In [1]:
import os

os.chdir('../..')

In [2]:
import glob
from os.path import basename, join, splitext

import numpy as np
import pandas as pd

import modules.point_processing as pp


In [3]:
def combine_dataframes(load_dir, trial_names):
    """Combine dataframes from different walking trials."""
    dataframe_dict = {}

    for trial_name in trial_names:
        dataframe_dict[trial_name] = pd.read_pickle(
            join(load_dir, trial_name) + '.pkl')

    return pd.concat(dataframe_dict)


In [4]:
kinect_dir = join('data', 'kinect')

df_truth = pd.read_pickle('results/dataframes/df_truth.pkl')
trial_names = df_truth.index.get_level_values(0).unique().values

df_hypo = combine_dataframes(
    join(kinect_dir, 'processed', 'hypothesis'), trial_names)
df_selected = combine_dataframes(join(kinect_dir, 'best_pos'), trial_names)
df_assigned = combine_dataframes(join(kinect_dir, 'assigned'), trial_names)

foot_parts = ['L_FOOT', 'R_FOOT']

# Truth positions on frames with head and both feet
df_truth = df_truth.loc[:, ['HEAD'] + foot_parts].dropna()

# Drop index level for walking pass
# This gives it the same index as the other dataframes
df_assigned.index = df_assigned.index.droplevel(1)

# %% Take frames with truth and selected positions

index_intersect = df_truth.index.intersection(df_selected.index)

df_truth = df_truth.loc[index_intersect]
df_hypo = df_hypo.loc[index_intersect]
df_selected = df_selected.loc[index_intersect]
df_assigned = df_assigned.loc[index_intersect]

In [5]:
file_paths = sorted(glob.glob(join(kinect_dir, 'best_pos_radii', '*.pkl')))

dataframe_dict = {}

for file_path in file_paths:
    file_name = splitext(basename(file_path))[0]
    trial_name, radius = file_name.split('_radius_')
    
    dataframe_dict[(int(radius), trial_name)] = pd.read_pickle(file_path)
   

df_radii = pd.concat(dataframe_dict)
df_radii.index.names = ['radius', 'trial_name', 'frame']

radii = df_radii.index.get_level_values(0).unique().values

In [6]:
proposals = df_hypo.FOOT.values

truth_l = np.stack(df_truth.L_FOOT)
truth_r = np.stack(df_truth.R_FOOT)

# Create modified truth
truth_mod_l = pp.closest_proposals(proposals, np.stack(df_truth.L_FOOT))
truth_mod_r = pp.closest_proposals(proposals, np.stack(df_truth.R_FOOT))

for radius in radii:

    df_selected = df_radii.loc[radius].loc[index_intersect]
    
    selected_l = np.stack(df_selected.L_FOOT)
    selected_r = np.stack(df_selected.R_FOOT)
    
    # Match selected positions with truth
    matched_l, matched_r = pp.match_pairs(selected_l, selected_r, truth_l,
                                          truth_r)
    
    acc_matched = pp.double_position_accuracy(
        matched_l, matched_r, truth_l, truth_r, max_dist=15)
    
    acc_matched_mod = pp.double_position_accuracy(
        matched_l, matched_r, truth_mod_l, truth_mod_r, max_dist=15)
    
    print(acc_matched, acc_matched_mod)

0.41480206540447506 0.5869191049913941
0.7986230636833046 0.9621342512908778
0.7865748709122203 0.9604130808950087
0.8055077452667814 0.9672977624784854
0.7951807228915663 0.9672977624784854
0.8089500860585198 0.9741824440619621
0.7934595524956971 0.9655765920826161
0.810671256454389 0.9724612736660929
0.8003442340791739 0.9690189328743546
0.810671256454389 0.9724612736660929
0.8003442340791739 0.9690189328743546
0.8003442340791739 0.9690189328743546
0.8003442340791739 0.9690189328743546
0.8003442340791739 0.9690189328743546
0.8003442340791739 0.9690189328743546
